# Moments calculation

## Introduction

Given a polygon that defines a closed path $\partial \Sigma$ containing the region $\Sigma$, we define the moment $M_{ij}$ as the integral:
$$
    M_{ij} = \int_{\Sigma} x^i y^j d^2\Sigma
$$
just like for the [OpenCV Moments](https://docs.opencv.org/3.4/d8/d23/classcv_1_1Moments.html) struct.

## Computation

To compute the moments, we want to use the [Stokes Theorem] to compute the integral of the polygon as a sum over the edges of the polygon.

In our case, we have
$$
    \frac{\partial F_y}{\partial x} - \frac{\partial F_x}{\partial y} = x^i y^j
$$
where $F_x$ and $F_y$ are the first fundamental forms of the polygon. (??)

Now, the [Stokes Theorem] states that the integral of the above expression over the polygon is equal to the line integral of the vector field $(F_y, F_x)$ around the polygon:
$$
    \int_{\Sigma} x^i y^j dx\,dy =
    \oint_{\partial \Sigma} \mathbf{F} \cdot d\mathbf{\ell} =
    \sum_{S \in \partial \Sigma} \int_{S} \mathbf{F} \cdot d\mathbf{\ell}
$$
Where $S$ is a segment of the polygon boundary.

The problem is now reduced to finding the vector field $\mathbf{F}$ and the line integral over one segment.

[Stokes Theorem]: https://en.wikipedia.org/wiki/Stokes%27_theorem

### Initial setup

In [ ]:
import sympy as sp                       # symbolic math
from sympy import Symbol, Expr, Function # for math types
from IPython.display import Math, Latex  # for displaying math
import numpy as np                       # for numerical math
import typing as tp                      # for typing

#sp.init_printing(use_latex='mathjax')
sp.init_printing()

Some constants

In [ ]:
# orders more that 4 could take a long time to compute
MAX_ORDER = 2

# The symbols used for the final solution simplification, higher orders might require more symbols
UTILITY_SYMBOLS=[sp.symbols('d' + str(i)) for i in range(1, 201)]

Here we define some symbols

In [ ]:
# variables
x, y = sp.symbols('x y')
t = sp.symbols('t')

# exponents
i, j = sp.symbols('i j', positive=True, integer=True)

# segment points
x0, y0 = sp.symbols('x_0 y_0')
x1, y1 = sp.symbols('x_1 y_1')
delta_x, delta_y = sp.symbols('Delta_x Delta_y')

### Solving the 1-form

We now want to find the 1-form by satisfying the following condition:
$$
\frac{\partial F_y}{\partial x} - \frac{\partial F_x}{\partial y} = x^i y^j
$$
There are two particularly simple solutions that can be found by imposing $F_x = 0$ and $F_y = 0$ respectively:

In [ ]:
integrand = x**i * y**j
sol_1 = sp.Matrix([[0], [sp.integrate(integrand, x)]])
sol_2 = sp.Matrix([[sp.integrate(-integrand, y)], [0]])
print('Solution 1')
display(sol_1)
print('Solution 2')
display(sol_2)

Sometimes, particular combinations of these two are useful, we guess that `(sol_1 + sol_2) / 2` is a good candidate.

In [ ]:
SOLUTIONS = [sol_1, sol_2, (sol_1 + sol_2) / 2]
display(SOLUTIONS)

We now have to integrate it over a segment of the curve.
$$
\int_S \mathbf{F} \cdot d\mathbf{\ell}
$$
Where $S$ is the segment of the curve.

We define an utility function to find the integral for a given $i$ and $j$.

In [ ]:
def integrate_over_segment(sol: Expr, x0: Expr, y0: Expr, x1: Expr, y1: Expr, i_val: int, j_val: int) -> Expr :
    xt = x0 + t * (x1 - x0)
    yt = y0 + t * (y1 - y0)
    delta_y = y1 - y0
    delta_x = x1 - x0

    sol = sol.subs(x, xt).subs(y, yt).subs(i, i_val).subs(j, j_val)
    #display(sol)

    integrand = sol[0, 0] * delta_x + sol[1, 0] * delta_y
    #display(integrand)

    integral = sp.integrate(integrand, (t, 0, 1))
    #display(integral)

    return integral

As an example, we can try to find the integrals for $i = j = 0$ and use the `factor` function to simplify the result.

> Note: useful functions: `simplify`, `collect`, `factor`, `cse`

In [ ]:
I1 = integrate_over_segment(sol_1, x0, y0, x1, y1, 0, 0).factor()
I2 = integrate_over_segment(sol_2, x0, y0, x1, y1, 0, 0).factor()
I3 = integrate_over_segment(SOLUTIONS[2], x0, y0, x1, y1, 0, 0).factor()
display(I1, I2, I3)

We notice that the two results are not the same, but the overall sum is the same. To numerically test this, we can take a bunch of points:

In [ ]:
def curve(t: float) -> tp.Tuple[float, float]:
    return (np.cos(t * 2 * np.pi), np.sin(t * 2 * np.pi))
PTS = np.array([curve(t) for t in np.linspace(0, 1, 100)])
def segment_integral_1(x0_value: float, y0_value: float, x1_value: float, y1_value: float) -> float:
    return I1.evalf(subs={x0: x0_value, y0: y0_value, x1: x1_value, y1: y1_value})
def segment_integral_2(x0_value: float, y0_value: float, x1_value: float, y1_value: float) -> float:
    return I2.evalf(subs={x0: x0_value, y0: y0_value, x1: x1_value, y1: y1_value})

def compute_area_1() -> float:
    area = 0
    for i in range(PTS.shape[0] - 1):
        area += segment_integral_1(PTS[i, 0], PTS[i, 1], PTS[i + 1, 0], PTS[i + 1, 1])
    return area
def compute_area_2() -> float:
    area = 0
    for i in range(PTS.shape[0] - 1):
        area += segment_integral_2(PTS[i, 0], PTS[i, 1], PTS[i + 1, 0], PTS[i + 1, 1])
    return area

print('Area 1:', compute_area_1())
print('Area 2:', compute_area_2())
print('pi    :', np.pi)

We can now build a table with all the integrals for each $i$ and $j$ of every solution of $\mathbf{F}$:

In [ ]:
def solution_matrix(sol: Expr, N: int) -> Expr:
    M = sp.zeros(N + 1, N + 1)
    for i in range(N + 1):
        for j in range(N + 1):
            I = integrate_over_segment(sol, x0, y0, x1, y1, i, j)
            I = I.factor()
            M[i, j] = I
    return M
def display_solution(sol: Expr, N: int):
    display(solution_matrix(sol, N))
print('Solutions for orders 0 to', MAX_ORDER)
for solution in SOLUTIONS:
    display_solution(solution, MAX_ORDER)

Unfortunately, none of this solutions looks particularly beautiful. Also, we have shown how the overall sum will be the same for each of this, but which is the best? Which of this gives the minimum error?

To answer this question, we can try to somehow remove the terms that will not contribute to the overall sum. We can do this by considering the following figure:

![polygon_manipulation](./polygon_manipulation.svg)

The blue contour is the polygon that we want to integrate. The line integral over the contour will compute the original integral over the surface $\Sigma$ delimited by the polygon.  
If we add a two overlapped segments $S_0$ and $-S_0$ to the contour (violet line) the integral over $\Sigma$ will be the same ase we are adding an null measure to the contour. This can be shown by observing that $I_{S_0} = -I_{-S_0}$. We can show this explicitly:

In [ ]:
I_S0 = integrate_over_segment(SOLUTIONS[0], x0, y0, x1, y1, 0, 0).factor()
I_mS0 = integrate_over_segment(SOLUTIONS[0], x1, y1, x0, y0, 0, 0).factor()
display(I_S0, I_mS0)
print(sp.Eq(I_S0, -I_mS0))

Now we could use this fact to try to remove the terms that will not contribute to the overall sum. In particular, we can recall that we have found the solutions by integrating the differential equation imposing $\mathbf{F}(0) = 0$, we have to remenber this in the next step.

We now insert a segment $S_0$ and $-S_0$ that touches the origin an extremity of a segment for each segment of the contour. The overall integral will be the same as before, but it is now written:
$$
\oint_{\partial \Sigma} = \sum \int_{S_{0, n}} + \int_{S_n} + \int_{-S_{0, n}} = \sum I_{S_{0, n}} + I_{S_n} + I_{-S_{0, n}}
$$
The fact that $\mathbf{F}(0) = 0$ will cancel out all the unwanted terms.

We also notice that this is equivalent to evaluate this integrals over a bunch of triangles connected to the origin and summing altogether.

In [ ]:
def solution_matrix_simplified(sol: Expr, N: int) -> Expr:
    M = solution_matrix(sol, N)
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            I = M[i, j]
            I_S0 = I.subs(x0, 0).subs(y0, 0).subs(x1, x0).subs(y1, y0)
            I_S = I
            I_mS0 = I.subs(x1, 0).subs(y1, 0).subs(x0, x1).subs(y0, y1)
            I = I_S0 + I_S + I_mS0
            M[i, j] = I.expand().simplify().factor()
    return M
M_SOLUTIONS = []
for solution in SOLUTIONS:
    M_SOLUTIONS.append(solution_matrix_simplified(solution, MAX_ORDER))
def are_same():
    for i in range(len(M_SOLUTIONS)):
        for j in range(i + 1, len(M_SOLUTIONS)):
            if not sp.Eq(M_SOLUTIONS[i], M_SOLUTIONS[j]):
                return False
    return True
print('Solutions are the same:', are_same())

In [ ]:
M_SOLUTIONS = M_SOLUTIONS[0]
print('Solution for orders 0 to', MAX_ORDER)
display(M_SOLUTIONS)

(defs, simplified) = sp.cse(M_SOLUTIONS, UTILITY_SYMBOLS)
print("simplified:")
display(simplified)
print("where:")
display(defs)

## Code generation

We now want to generate some useful code from the above results.

First, we write the code for the defines:

In [ ]:
for definition in defs:
    symbol, value = definition
    print(sp.ccode(symbol), '=', sp.ccode(value))

An then we can generate the code for the integrals:

In [ ]:
for i in range(simplified[0].shape[0]):
    for j in range(simplified[0].shape[1]):
        print('I[', i, '][', j, '] = ', sp.ccode(simplified[0][i, j]), ';', sep='')